In [62]:
'''
In this script we are going to try out LDA topic modelling using gensim.
THIS IS REALLY COOL!
https://www.youtube.com/watch?v=UEn3xHNBXJU&ab_channel=PythonTutorialsforDigitalHumanities
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#9createbigramandtrigrammodels

more resources:
https://towardsdatascience.com/basic-nlp-on-the-texts-of-harry-potter-topic-modeling-with-latent-dirichlet-allocation-f3c00f77b0f5
https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28 
# https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
'''

'\nIn this script we are going to try out LDA topic modelling using gensim.\nTHIS IS REALLY COOL!\n'

In [81]:
import numpy as np
import pandas as pd
import glob
 
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel

#spacy
import spacy

#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [166]:
df1 = pd.read_csv("data/new_york_times/NYT_clean_1.csv")
df2 = pd.read_csv("data/new_york_times/NYT_clean_2.csv")
df3 = pd.read_csv("data/new_york_times/NYT_clean_3.csv")

df = pd.concat([df1, df2, df3])

In [154]:
print(df.iloc[1,4])
print(df.info())

year frustration United Nations effort bring peace Afghanistan reach critical point today envoy begin round shuttle diplomacy aim bring main warring group new coalition Government United Nations representative Mahmoud Mestiri hope new govern council power Kabul capital Monday open talk Mr. Mestiri say early date slip Tuesday Afghans involve talk judge prospect meet deadline increasingly slim peace effort snag thing bring rapid momentum Mr. Mestiri effort recent week sudden military success new afghan fighting force know Taliban month take control nearly half country offensive begin Kandahar Afghanistan second large city Taliban sprang student muslim religious school band fall rid country armed faction divide county fief prey ordinary Afghans week Taliban unit push gate Kabul halt apparently review tactic deal Government President Burnahuddin Rabbani arrive Friday Mr. Mestiri acknowledge Taliban offensive change political landscape way alter doom plan peace effort close success time beg

In [155]:
#break it down by individual words
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text)
        final.append(new)
    return (final)

In [157]:
data_words = gen_words(df["bread_text_preprocessed"])

In [158]:
print("Before applying the helper function gen_words: \n" + str(df.iloc[0,4][0:142]))
print("\nAfter applying the helper function gen_words: \n" + str(data_words[0][0:20]))

Before applying the helper function gen_words: 
decade soviet military occupation year ruthless civil war Afghanistan take sudden startling turn peace military campaign last barely month new

After applying the helper function gen_words: 
['decade', 'soviet', 'military', 'occupation', 'year', 'ruthless', 'civil', 'war', 'afghanistan', 'take', 'sudden', 'startling', 'turn', 'peace', 'military', 'campaign', 'last', 'barely', 'month', 'new']


# Bigrams and Trigrams

In [159]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=80)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=80)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [160]:
print("Without bigrams and trigrams:\n" + str(data_words[1067][0:100]))
print ("\nWith bigrams and trigrams:\n" + str(data_bigrams_trigrams[1067][0:100]))

Without bigrams and trigrams:
['bin', 'laden', 'whereabouts', 'afghanistan', 'rule', 'taliban', 'conflicting', 'statement', 'whereabout', 'osama', 'bin', 'laden', 'know', 'page', 'warnings', 'attacks', 'secretary', 'defense', 'donald', 'rumsfeld', 'administration', 'official', 'step', 'warning', 'threat', 'chemical', 'biological', 'nuclear', 'attack', 'page', 'beefing', 'surveillance', 'pentagon', 'concern', 'shortage', 'surveillance', 'equipment', 'pursuit', 'osama', 'bin', 'laden', 'step', 'purchase', 'reconnaissance', 'plane', 'page', 'time', 'mourner', 'woman', 'husband', 'found', 'banking', 'firm', 'lose', 'employee', 'world', 'trade', 'center', 'disaster', 'spend', 'day', 'attend', 'memorial', 'service', 'page']

With bigrams and trigrams:
['bin_laden_whereabouts', 'afghanistan', 'rule', 'taliban', 'conflicting', 'statement', 'whereabout_osama_bin', 'laden', 'know', 'page', 'warnings', 'attacks', 'secretary', 'defense', 'donald_rumsfeld', 'administration', 'official', 'step', 'wa

# TF-IDF removal

In [161]:
#removing frequently occuring words such as say. removing words that are generic and dont add any meaningful to our topics. 

texts = data_bigrams_trigrams

id2word = corpora.Dictionary(texts)

corpus = [id2word.doc2bow(text) for text in texts]

#dont know what this is
tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [162]:
corpus[0]
#first item is index of word, second item is frequency of the word

[(0, 1),
 (2, 1),
 (3, 1),
 (7, 6),
 (10, 1),
 (12, 1),
 (16, 1),
 (19, 2),
 (25, 1),
 (26, 1),
 (27, 2),
 (29, 1),
 (31, 1),
 (32, 1),
 (34, 1),
 (35, 1),
 (37, 1),
 (39, 1),
 (42, 1),
 (45, 3),
 (51, 1),
 (52, 1),
 (53, 5),
 (55, 1),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 3),
 (62, 3),
 (63, 2),
 (70, 1),
 (72, 2),
 (73, 1),
 (74, 1),
 (75, 2),
 (77, 4),
 (79, 1),
 (81, 1),
 (84, 1),
 (86, 1),
 (87, 1),
 (88, 2),
 (90, 1),
 (91, 1),
 (92, 1),
 (95, 2),
 (96, 1),
 (98, 4),
 (107, 2),
 (108, 1),
 (109, 1),
 (110, 3),
 (111, 1),
 (112, 11),
 (114, 1),
 (115, 1),
 (116, 2),
 (118, 1),
 (120, 2),
 (122, 1),
 (123, 1),
 (125, 6),
 (127, 1),
 (128, 1),
 (130, 1),
 (132, 1),
 (135, 4),
 (137, 1),
 (138, 1),
 (140, 1),
 (142, 2),
 (143, 1),
 (144, 1),
 (147, 1),
 (148, 1),
 (149, 1),
 (150, 2),
 (153, 9),
 (154, 1),
 (155, 11),
 (156, 4),
 (157, 1),
 (159, 1),
 (163, 6),
 (169, 1),
 (170, 1),
 (174, 2),
 (176, 2),
 (177, 3),
 (179, 2),
 (181, 1),
 (182, 1),
 (186, 1),
 (187, 2),
 (189, 6),
 (190

In [ ]:
# so now we have bigrams and trigrams and we have removed those words such as "say" that appear all the time. This should make better clustering.

# Creating a basic LDA topic model

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every = 1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha = "auto",
                                            )

In [34]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds = "mmds", R=30)
vis

C:\Users\ah140\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.380899  0.094004       1        1  19.104876
5     -0.023261  0.184734       2        1  15.496127
6     -0.246056  0.318051       3        1  14.581744
4      0.191007 -0.152741       4        1  13.524034
8      0.442450 -0.087580       5        1   9.343123
1      0.304973  0.202121       6        1   8.641479
7      0.110034  0.425698       7        1   7.811621
3      0.106729 -0.450869       8        1   4.279170
2     -0.376277 -0.234724       9        1   4.275708
9     -0.128701 -0.298694      10        1   2.942119, topic_info=          Term          Freq         Total Category  logprob  loglift
513      woman   9782.000000   9782.000000  Default  30.0000  30.0000
1       afghan  16082.000000  16082.000000  Default  29.0000  29.0000
80       kabul  14589.000000  14589.000000  Default  28.0000  28.0000
309   pakistan  10088.000000  10088.000000  Default  27.0000  27.0000
594      group   9463.000000   9463.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
204      claim    638.047508   1860.456487  Topic10  -4.6810   2.4559
1719    accuse    467.639208    927.126068  Topic10  -4.9918   2.8416
956       home    679.700262   3468.636182  Topic10  -4.6178   1.8962
1767    office    521.168108   3348.821966  Topic10  -4.8834   1.6657
1093   witness    426.694190   1009.003843  Topic10  -5.0834   2.6654

[526 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
514        7  0.998904         abc
16529      8  0.994269  aboriginal
1719       2  0.263179      accuse
1719       3  0.067952      accuse
1719       5  0.051773      accuse
...      ...       ...         ...
11198      5  0.999212   yorkshire
1713       4  0.711201       young
1713       9  0.288541       young
31643      4  0.998731   yousafzai
44091      9  0.997179       zahra

[766 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 7, 5, 9, 2, 8, 4, 3, 10])

# Coherence Score and choosing number of topics

In [35]:
# Compute Coherence Score for the current model with 10 topics
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4704628056474351


In [ ]:
# Nexp up, we are going to compute coherence scores for each number of topic

In [36]:
def compute_coherence_values(k):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
    return coherence_model_lda.get_coherence()

In [37]:
# Topics range
min_topics = 1
max_topics = 30
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

#empty dataframe
model_results = {'Topics': [],
                 'Coherence': []
                }

# iterate through number of topics
for k in topics_range:
    print(k)
    # get the coherence score for the given topics
    cv = compute_coherence_values(k=k)
    # Save the model results
    model_results['Topics'].append(k)
    model_results['Coherence'].append(cv)
                    


1
3
5
7
9
11
13
15
17
19
21
23
25
27
29


In [38]:
pd.DataFrame(model_results).to_csv('data/lda_topic_model/tuning_results_guardian.csv', index=False)

In [65]:
#plotting in R

# Final Models

In [91]:
lda_model_nyt = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 7,
                                            random_state = 100,
                                            update_every = 1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha = "auto",
                                            )

In [92]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model_nyt, corpus, id2word, mds = "mmds", R=30)
vis

C:\Users\ah140\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.111500 -0.004400       1        1  28.541801
0     -0.061896 -0.377768       2        1  17.613436
6     -0.393528 -0.155199       3        1  16.416045
5     -0.250797  0.307496       4        1  12.321575
1      0.282361 -0.265773       5        1  10.140131
2      0.354354  0.082782       6        1   9.915123
4      0.181006  0.412863       7        1   5.051889, topic_info=             Term           Freq          Total Category  logprob  loglift
189            mr  117064.000000  117064.000000  Default  30.0000  30.0000
430          kill   54778.000000   54778.000000  Default  29.0000  29.0000
6344        trump   33825.000000   33825.000000  Default  28.0000  28.0000
26262       biden   30377.000000   30377.000000  Default  27.0000  27.0000
203       officer   27207.000000   27207.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
3851        image     926.309827    1552.792309   Topic7  -5.7475   2.4688
1339       direct    1011.947457    2820.132976   Topic7  -5.6591   1.9605
1787   journalist    1042.025193    4248.208605   Topic7  -5.6298   1.5801
670          life    1042.352202    4734.182484   Topic7  -5.6295   1.4721
2304     director     917.480225    2414.755682   Topic7  -5.7571   2.0177

[405 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
5472       3  0.089938        abdullah
5472       5  0.909573        abdullah
15066      2  0.999141        activist
1613       1  0.211117  administration
1613       4  0.655630  administration
...      ...       ...             ...
366        1  0.999570            year
12416      5  0.999673           yemen
10612      5  0.999582          yemeni
1133       6  0.901234           young
1133       7  0.098585           young

[596 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 7, 6, 2, 3, 5])

In [101]:
lda_model_guardian = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 7,
                                            random_state = 100,
                                            update_every = 1,
                                             chunksize = 100,
                                            passes = 10,
                                            alpha = "auto",
                                            )

In [102]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model_guardian, corpus, id2word, mds = "mmds", R=30)
vis

C:\Users\ah140\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.035391  0.038870       1        1  29.110793
4     -0.110154 -0.365982       2        1  15.883376
1     -0.339878 -0.099174       3        1  14.551442
5     -0.278568  0.237983       4        1  12.762358
2      0.279118 -0.287479       5        1  11.202670
6      0.344926  0.074798       6        1   8.610882
3      0.139947  0.400984       7        1   7.878479, topic_info=           Term          Freq         Total Category  logprob  loglift
309    pakistan  11520.000000  11520.000000  Default  30.0000  30.0000
1        afghan  11619.000000  11619.000000  Default  29.0000  29.0000
594       group  10737.000000  10737.000000  Default  28.0000  28.0000
80        kabul  11766.000000  11766.000000  Default  27.0000  27.0000
513       woman  10678.000000  10678.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1374   violence    674.113877   1774.791650   Topic7  -5.6111   1.5730
1816  community    661.501919   1924.426314   Topic7  -5.6299   1.4732
23         city    591.415193   3461.310856   Topic7  -5.7419   0.7742
1091       west    538.206155   1662.257856   Topic7  -5.8362   1.4133
3470      write    531.663534   1663.541538   Topic7  -5.8484   1.4003

[419 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
514        5  0.998748         abc
16529      7  0.997619  aboriginal
1599       1  0.999053     achieve
7407       7  0.996780    activism
3989       2  0.583333    activist
...      ...       ...         ...
3471       4  0.998922       yemen
11198      5  0.999330   yorkshire
1713       2  0.999666       young
31643      2  0.999074   yousafzai
44091      2  0.998321       zahra

[640 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 6, 3, 7, 4])

# Grab dominant topic for each article

In [167]:
all_topics = []

#looping through all the articles in df
for i in range(len(df_nyt)):
    #getting the probability and index of each topic for given article 
    top_topics = lda_model_nyt.get_document_topics(corpus[i], minimum_probability=0.0)
    
    #checking if there was some lists that were not functioning correctly
    if len(top_topics) != 7:
        print("List of Topics is not 7 in article number " + str(i))
    
    #removing the index of the topic, keep only the probabilities
    topic_vec_prop = [top_topics[i][1] for i in range(7)]
    all_topics.append(topic_vec_prop)

In [170]:
#for each article, find the topic with the highest probability and find the probability of that topic.

dominant_topic = []
topic_contribution = []

for i in all_topics:
    max_prop = max(i)
    max_index = i.index(max_prop)
    
    dominant_topic.append(max_index)
    topic_contribution.append(max_prop)

In [176]:
#making new columns in df for dominant_topic and topic_contribution respectively
df['dominant_topic'] = dominant_topic
df['topic_contribution'] = topic_contribution

In [180]:
#splitting into three datasets
df1 = df.iloc[0:7500,]
df2 = df.iloc[7501:15000,]
df3 = df.iloc[15001:21109, ]

#saving to three new files
df1.to_csv("data/new_york_times/NYT_clean_1.csv", index = False)
df2.to_csv("data/new_york_times/NYT_clean_2.csv", index = False)
df3.to_csv("data/new_york_times/NYT_clean_3.csv", index = False)